This script is aiming to find the proper script to make a classicification of the images.

# Import pacakage

In [30]:
import tensorflow as tf
import numpy as np

import warnings # current version of seaborn generates a bunch of warnings that we'll ignore
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import imshow
import numpy as np
import PIL
#import requests
from io import BytesIO
from IPython.display import display
%matplotlib inline

import os
import scipy

# Use in the model
from tensorflow.keras.layers import Dense, Dropout

# Load Images

In [31]:
# Get the name of all the labels
home_path = '/home/bowen/Documents/Ecomed/'
data_path = '/home/bowen/Documents/Ecomed/dataset'
class_names = os.listdir(data_path)
class_names = ["pharmaceutical", "sharps", "trace_chemo"]

In [32]:
# Set up data generators that can read images from our dataset into Keras.
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.5)

train_generator = datagen.flow_from_directory(
        data_path,
        target_size=(224, 224),
        batch_size=24,
        class_mode='binary',
        subset='training')

validation_generator = datagen.flow_from_directory(
        data_path,
        target_size=(224, 224),
        batch_size=24,
        class_mode='binary',
        subset='validation')

Found 3003 images belonging to 3 classes.
Found 3001 images belonging to 3 classes.


In [8]:
# # Show several images
# data, labels = validation_generator.next()

# for i, sample in enumerate(data):
#     plt.imshow(sample)
#     print(class_names[int(labels[i])])
#     plt.show()

# MobileNet Model

In [33]:
# Import mobilenet
mobilenet = tf.keras.applications.MobileNet(include_top=False, input_shape=[224, 224, 3], weights='imagenet')
mobilenet.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
__________

In [34]:
# Add new output layers.
x = mobilenet.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
preds = tf.keras.layers.Dense(3, activation='softmax')(x)

# Combine feature layers with our new output layers.
model = tf.keras.Model(inputs=mobilenet.input, outputs=preds)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
__________

# Train the Model

In [35]:
# defintion of optimizer
# opt = tf.keras.optimizers.SGD(0.001)
opt = tf.train.GradientDescentOptimizer(0.001)
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [36]:
# definition of checkpoint
checkpoint_path = os.path.join(home_path + 'models', "model_success.h5")
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_best_only=True, save_weights_only=True)

In [6]:
#model = tf.keras.models.load_model(os.path.join(home_path + 'models', "model_new_layer.h5"))

In [37]:
# train model
with tf.device('/gpu:3'):
    model.fit_generator(generator=train_generator,
                        validation_data=validation_generator, 
                        epochs=500,
                        callbacks=[cp_callback])

Epoch 1/500
126/126 [==============================] - 93s 736ms/step - loss: 1.1731 - acc: 0.4693 - val_loss: 0.8963 - val_acc: 0.5721
Epoch 2/500
126/126 [==============================] - 82s 652ms/step - loss: 0.8735 - acc: 0.6058 - val_loss: 0.6994 - val_acc: 0.7061
Epoch 3/500
126/126 [==============================] - 83s 661ms/step - loss: 0.6100 - acc: 0.7483 - val_loss: 0.5593 - val_acc: 0.7481
Epoch 4/500
126/126 [==============================] - 83s 662ms/step - loss: 0.4484 - acc: 0.8281 - val_loss: 0.4698 - val_acc: 0.7871
Epoch 5/500
126/126 [==============================] - 82s 650ms/step - loss: 0.3098 - acc: 0.8806 - val_loss: 0.4651 - val_acc: 0.8027
Epoch 6/500
126/126 [==============================] - 84s 664ms/step - loss: 0.2370 - acc: 0.9183 - val_loss: 0.3915 - val_acc: 0.8354
Epoch 7/500
126/126 [==============================] - 83s 657ms/step - loss: 0.1829 - acc: 0.9431 - val_loss: 0.3640 - val_acc: 0.8437
Epoch 8/500
126/126 [===========================

Epoch 61/500
126/126 [==============================] - 83s 660ms/step - loss: 0.0142 - acc: 0.9957 - val_loss: 0.1663 - val_acc: 0.9580
Epoch 62/500
126/126 [==============================] - 83s 660ms/step - loss: 0.0129 - acc: 0.9974 - val_loss: 0.1932 - val_acc: 0.9487
Epoch 63/500
126/126 [==============================] - 84s 664ms/step - loss: 0.0150 - acc: 0.9950 - val_loss: 0.1789 - val_acc: 0.9550
Epoch 64/500
126/126 [==============================] - 82s 653ms/step - loss: 0.0250 - acc: 0.9934 - val_loss: 0.1646 - val_acc: 0.9597
Epoch 65/500
126/126 [==============================] - 83s 658ms/step - loss: 0.0400 - acc: 0.9924 - val_loss: 0.1679 - val_acc: 0.9533
Epoch 66/500
126/126 [==============================] - 83s 656ms/step - loss: 0.0155 - acc: 0.9940 - val_loss: 0.1962 - val_acc: 0.9507
Epoch 67/500
126/126 [==============================] - 83s 660ms/step - loss: 0.0370 - acc: 0.9944 - val_loss: 0.1685 - val_acc: 0.9557
Epoch 68/500
126/126 [===================

126/126 [==============================] - 82s 651ms/step - loss: 0.0129 - acc: 0.9941 - val_loss: 0.1781 - val_acc: 0.9550
Epoch 121/500
126/126 [==============================] - 86s 685ms/step - loss: 0.0062 - acc: 0.9990 - val_loss: 0.1394 - val_acc: 0.9667
Epoch 122/500
126/126 [==============================] - 86s 682ms/step - loss: 0.0060 - acc: 0.9980 - val_loss: 0.1406 - val_acc: 0.9673
Epoch 123/500
126/126 [==============================] - 82s 650ms/step - loss: 0.0049 - acc: 0.9983 - val_loss: 0.1378 - val_acc: 0.9680
Epoch 124/500
126/126 [==============================] - 83s 662ms/step - loss: 0.0106 - acc: 0.9974 - val_loss: 0.1302 - val_acc: 0.9693
Epoch 125/500
126/126 [==============================] - 84s 664ms/step - loss: 0.0133 - acc: 0.9921 - val_loss: 0.1610 - val_acc: 0.9597
Epoch 126/500
126/126 [==============================] - 84s 663ms/step - loss: 0.0171 - acc: 0.9954 - val_loss: 0.1366 - val_acc: 0.9687
Epoch 127/500
126/126 [=========================

KeyboardInterrupt: 

In [38]:
tf.contrib.saved_model.save_keras_model(model, home_path + 'models')

INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO:tensorflow:Signatures INCLUDED in export for Train: ['train']
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: ['eval']
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tens

b'/home/bowen/Documents/Ecomed/models/1551769465'